In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#загрузим нашу финальную модель с нашей репы в HuggingFace, куда до этого мы ее сохранили после обучения
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "Gnider/hack_final_30ep_xmlroberta_aug9000"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [2]:
import pandas as pd

# загружаем тестовые данные и соединяем assessment, tags и text (как и в блокноте с обучением)
df_test = pd.read_csv('/kaggle/input/testes/test.csv(3).csv')

def preprocess_text_tags_and_assessment(row):
    text = row['text'] if isinstance(row['text'], str) else ""
    tags = row['tags'].replace('{', '').replace('}', '').replace(',', ' ') if isinstance(row['tags'], str) else ""
    assessment = str(row['assessment']) if pd.notnull(row['assessment']) else ""
    return f"{text.lower()} {tags.lower()} {assessment}"

df_test['combined_text'] = df_test.apply(preprocess_text_tags_and_assessment, axis=1)


In [3]:
df_test

,Unnamed: 0,index,assessment,tags,text,combined_text
0,1,3135,3.0,{DELIVERY},"Последнее время думаю плохо, сроки доставки да...","последнее время думаю плохо, сроки доставки да..."
1,3,4655,2.0,"{PRICE,DELIVERY,ASSORTMENT}",Цены намного выше магазинных но радуют акции,цены намного выше магазинных но радуют акции p...
2,5,22118,2.0,"{CATALOG_NAVIGATION,ASSORTMENT,DELIVERY}","Доставка за [NUM] минут, заказ даже не начали ...","доставка за [num] минут, заказ даже не начали ..."
3,7,23511,0.0,{DELIVERY},Ужасно долгая доставка,ужасно долгая доставка delivery 0.0
4,8,45,6.0,"{ASSORTMENT,PROMOTIONS}",Добрый вечер! Вы большие молодцы. Меня всё уст...,добрый вечер! вы большие молодцы. меня всё уст...
...,...,...,...,...,...,...
9010,16992,3523,3.0,"{PRICE,SUPPORT,DELIVERY}",Задержка с доставкой не даете промокод на скид...,задержка с доставкой не даете промокод на скид...
9011,16993,24925,6.0,"{PRICE,PRODUCTS_QUALITY,ASSORTMENT}",Очень удобный формат сервиса и очень маленький...,очень удобный формат сервиса и очень маленький...
9012,16994,6327,6.0,"{PAYMENT,ASSORTMENT,DELIVERY}","Сумма заказа почти всегда высокая, что зачасту...","сумма заказа почти всегда высокая, что зачасту..."
9013,16997,530,3.0,"{PRODUCTS_QUALITY,SUPPORT,DELIVERY}","Часто, заказываю у вас молочную продукцию, при...","часто, заказываю у вас молочную продукцию, при..."


In [4]:
df_test['combined_text'][2]

'доставка за [num] минут, заказ даже не начали собирать спустя [num] минут catalog_navigation assortment delivery 2.0'

In [5]:
# токенизируем тексты
# Здесь хочется отметить, что как в блокноте с обучением, так и здесь, максимальная длина выставлена на 512. Видимо, от волнения и желания не опоздать с дедлайном я пропустил, что надо было бы проверить, какая там максимальная длина у сэмплов, и что-то предпринять. Но уже как есть, и результат вроде неплохой.
inputs = tokenizer(df_test['combined_text'].tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')

In [6]:
from tqdm import tqdm
import torch

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Function to predict in batches
def predict_in_batches(model, inputs, batch_size=16):
    predictions = []
    for i in tqdm(range(0, len(inputs['input_ids']), batch_size)):
        # Slice the inputs to get a batch
        batch_inputs = {k: v[i:i + batch_size].to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**batch_inputs)
            
        # Convert logits to binary predictions
        batch_predictions = (outputs.logits > 0).int().cpu().numpy()
        predictions.extend(batch_predictions)
    
    return predictions

# Run prediction in batches and ensure the inputs are on the correct device
inputs = tokenizer(df_test['combined_text'].tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to device
predictions = predict_in_batches(model, inputs, batch_size=16)

100%|██████████| 564/564 [01:33<00:00,  6.06it/s]


In [7]:
df_test['predicted_classes'] = predictions

In [8]:
def transform_prediction_to_string(prediction):
    class_indices = [str(idx) for idx, val in enumerate(prediction) if val == 1]
    return " ".join(class_indices) if class_indices else ""

df_test['target'] = df_test['predicted_classes'].apply(transform_prediction_to_string)

print(df_test)

      Unnamed: 0  index  assessment                                      tags  \
0              1   3135         3.0                                {DELIVERY}   
1              3   4655         2.0               {PRICE,DELIVERY,ASSORTMENT}   
2              5  22118         2.0  {CATALOG_NAVIGATION,ASSORTMENT,DELIVERY}   
3              7  23511         0.0                                {DELIVERY}   
4              8     45         6.0                   {ASSORTMENT,PROMOTIONS}   
...          ...    ...         ...                                       ...   
9010       16992   3523         3.0                  {PRICE,SUPPORT,DELIVERY}   
9011       16993  24925         6.0       {PRICE,PRODUCTS_QUALITY,ASSORTMENT}   
9012       16994   6327         6.0             {PAYMENT,ASSORTMENT,DELIVERY}   
9013       16997    530         3.0       {PRODUCTS_QUALITY,SUPPORT,DELIVERY}   
9014       16998  20221         6.0    {ASSORTMENT,PRICE,PROMOTIONS,DELIVERY}   

                           

In [9]:
df_done = df_test[['index', 'target']]

In [10]:
df_done

,index,target
0,3135,1 2 12
1,4655,12
2,22118,35
3,23511,0
4,45,0 16
...,...,...
9010,3523,37
9011,24925,12 22 28
9012,6327,8
9013,530,15


In [11]:
df_done.to_csv('/kaggle/working/test_pred.csv', encoding = 'utf-8', index=False)